In [ ]:
# IMPORTS 
import pandas as pd

In [ ]:
# CHARGEMENT TITLE_BASICS
title_basics = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False)

In [ ]:
# Filtrage titleType = movie seulement
titlebasicsmovies = title_basics.loc[title_basics['titleType'].isin(['movie', 'tvSeries'])]
# titlebasicsmovies['titleType'].unique() CONFIRME

# Filtrage genres = sans \N
titlebasicsmovies1 = titlebasicsmovies.loc[titlebasicsmovies['genres'] != r'\N']

# Filtrage runtimeMinutes = sans \N
titlebasicsmovies2 = titlebasicsmovies1.loc[titlebasicsmovies1['runtimeMinutes'] != r'\N']

# Suppression colonnes
titlebasicsmovies3 = titlebasicsmovies2.drop(columns = ['endYear', 'isAdult', 'originalTitle'])

# Filtrage primary_title sans \N et remplacement par le bon nom de film
titlebasicsmovies4 = titlebasicsmovies3.loc[titlebasicsmovies3['primaryTitle'] != r'NaN']
titlebasicsmovies4.loc[titlebasicsmovies4['primaryTitle'].isna(), 'primaryTitle'] = 'None'

# Filtrage par genres
dfa = titlebasicsmovies4[
    titlebasicsmovies4['genres'].apply(lambda g: 
        'Comedy' in g or 'Action' in g or 'Thriller' in g)]

In [ ]:
# CHARGEMENT TMDB FULL
tmdb = pd.read_csv('tmdb_full.csv', sep=',', low_memory=False)

In [ ]:
# MERGE de TMDB_FULL avec title_basics_Ange (Ange)

df= pd.merge(dfa,
              tmdb, 
              how= 'left',
              left_on = 'tconst',
              right_on = 'imdb_id')

In [ ]:
# FILTRAGE pour enlever les lignes avec NaN ==> 75 566 rows
df_sans_nan = df.dropna(subset=[
    'overview',
    'popularity',
    'poster_path',
    'production_countries',
    'release_date',
    'runtime',
    'vote_average',
    'vote_count',
    'title'
])

In [ ]:
# FILTRAGE ===> 10034 rows
# FILTRAGE pour le statut == released
df_sans_nan1 = df_sans_nan.loc[df_sans_nan['status'] == 'Released']
# FILTRAGE par les 2 productions_countries appréciés
df_sans_nan2 = df_sans_nan1[df_sans_nan1['production_countries'].apply(lambda c: 'FR' in c or 'US' in c)]
# FILTRAGE par les films ayant plus de 6.5/10 de notes
df_sans_nan3 = df_sans_nan2[df_sans_nan2['vote_average'] >= 6.01]

In [ ]:
tmdb_df = df_sans_nan3.drop(columns=['Unnamed: 0', 'titleType', 'backdrop_path', 'adult', 'budget', 'genres_y', 'homepage', 'original_language', 'original_title', 'revenue', 'runtime', 'spoken_languages',
                    'tagline', 'production_companies_name', 'production_companies_country', 'video', 'status', 'primaryTitle'])

In [ ]:
# Chargement des fichiers
principals_df = pd.read_csv("title.principals.tsv", sep='\t', na_values='\\N')
name_basics_df = pd.read_csv("name.basics.tsv", sep='\t', na_values='\\N')

In [ ]:
# filtrer les acteurs dans principals
principals_df = principals_df[principals_df['category'].isin(['actor', 'actress'])]
# filtrer le name_basics_df avec le nconst et les noms des acteurs
df_nb = name_basics_df[['nconst', 'primaryName']]

In [ ]:
# joindre principals avec name_basics pour récupérer les noms
merged_df = principals_df.merge(df_nb, on='nconst', how='left')
# garder que les films présents dans tmdb
merged_df = merged_df[merged_df['tconst'].isin(tmdb_df['imdb_id'])]

In [ ]:
# grouper les noms d’acteurs par film
actors_by_film = merged_df.groupby('tconst')['primaryName'].apply(list).reset_index()
# renommer tconst en imdb_id pour join final
actors_by_film = actors_by_film.rename(columns={'tconst': 'imdb_id'})
# merge final
final_df = tmdb_df.merge(actors_by_film, on='imdb_id', how='left')
# renommer la colonne
final_df = final_df.rename(columns={'primaryName': 'actors'})
# suppression des colonnes inutiles
final = final_df.drop(columns=['Unnamed: 0', 'id'])

In [ ]:
# réordonner les colonnes
colonnes = [
    'title',
    'release_year',
    'genres_x',
    'runtimeMinutes',
    'production_countries',
    'actors',
    'vote_average',
    'vote_count',
    'popularity',
    'poster_path',
    'imdb_id',
    'overview',
    'id'
]

final_cleaned = final_cleaned[colonnes]

In [ ]:
final_cleaned['overview'] = final_cleaned['overview'].str.replace(r'[\r\n]+', ' ', regex=True)

In [ ]:
# SPLITTER LA COLONNE GENRES_X
# Séparer la colonne "genres" en plusieurs colonnes
genres_split = df['genres_x'].str.split(',', expand=True)

# Renommer les nouvelles colonnes si souhaité
genres_split.columns = [f'genre_{i+1}' for i in range(genres_split.shape[1])]

# Fusionner avec le DataFrame d'origine (sans la colonne "genres" initiale si besoin)
df = df.drop(columns='genres_x').join(genres_split)

In [ ]:
# SPLITTER LA COLONNE PRODUCTION_COUNTRIES
# Séparer la colonne "genres" en plusieurs colonnes
nationality_split = df['production_countries'].str.split(',', expand=True)

# Renommer les nouvelles colonnes si souhaité
nationality_split.columns = [f'nationality_{i+1}' for i in range(nationality_split.shape[1])]

# Fusionner avec le DataFrame d'origine (sans la colonne "genres" initiale si besoin)
df = df.drop(columns='production_countries').join(nationality_split)

In [ ]:
df = df.drop(columns= ['nationality_4', 'nationality_5', 'nationality_6', 'nationality_7', 'nationality_8', 'nationality_9', 'nationality_10', 'nationality_11', 'nationality_12', 'nationality_13', 'nationality_14'])

In [ ]:
# SPLITTER LA COLONNE ACTORS en 5 colonnes

# Retirer les crochets et découper la chaîne
df['actors_list'] = df['actors'].str.strip("[]").str.replace("'", "").str.split(", ")

df[['actor_1', 'actor_2', 'actor_3', 'actor_4', 'actor_5']] = df['actors_list'].apply(lambda x: pd.Series(x[:5]))

In [ ]:
df.to_csv("dataset_final_splitted.csv")